In [1]:
#import libraries
import boto3
import json
import time

In [3]:
# Connection to AWS
folder = '/home/propietari/Documents/claus/' # TODO: change to your local path
file_name = 'AWS_S3_keys_JordiPlanas_Made_in_game.json' # TODO: Change to your filename
folder = 'C:/Users/jordi/Documents/claus/' # TODO: change to your local path
file_name = 'AWS_S3_keys_wri.json' # TODO: Change to your filename
file = folder + file_name

with open(file, 'r') as dict:
    credentials = json.load(dict)
                                      
KEY = list(credentials)[0]
SECRET = list(credentials.values())[0]
# s3BucketName = "wri-testing"
s3BucketName = "wri-nlp-policy"
# region = 'eu-central-1'
region = "us-east-1"

s3 = boto3.resource(
    service_name = 's3',
    region_name = region,
    aws_access_key_id = KEY,
    aws_secret_access_key = SECRET
)

In [8]:
# prefix = "english_documents/raw_pdf/"
# i = 0
# for object in s3.Bucket(s3BucketName).objects.all().filter(Prefix=prefix):
#     srcKey = object.key
#     if ".txt" in srcKey:
#         print(i)
#         newKey = srcKey.replace(".txt", ".pdf")
#         copySource = s3BucketName + '/' + srcKey         
#         s3.Object(s3BucketName, newKey).copy_from(CopySource=copySource)
#         s3.Object(s3BucketName, srcKey).delete()
#     i += 1

3
5
6
8
9
11
12
13
14
15
17
20
22
23
25
26
27
28
29
30
31
38
39
41
42
43
44
45
46
47
48
49
50
54
57
58
59
61
62
63
69
73
75
76
77
80
81
82
83
84
85
86
87
88
89
95
97
98
101
103
104
106
107
108
110
111
112
113
114
115
116
118
119
120
123
124
126
128
131
135
136
138
139
140
142
143
144
146
150
151
155
163
165
166
168
170
171
174
175
180
181
184
185
186
188
189
191
192
194
195
197
198
199
201
202
203
204
205
206
209
210
213
217
221
222
224
228
229
231
236
237
239
240
243
244
245
246
247
250
251
252
256
257
260
266
267
268
269
270
271
273
274
276
279
280
281
283
284
285
286
291
292
294
300
301
302
304
306
307
309
312
313
315
316
319
320
322
323
324
325
326
327
328
331
332
333
335
336
337
339
340
341
342
345
346
347
352
354
356
358
359
361
363
368
369
370
373
374
376
377
378
379
380
382
386
387
390
393
399
400
404
405
406
407
408
409
410
411
412
413
418
419
420
421
422
425
426
427
429
431
432
434
435
438
439
440
442
443
444
449
453
455
456
457
458
460
464
469
470
472
473
474
475
476
477
480

In [4]:
def startJob(s3BucketName, objectName):
    response = None
    client = boto3.client('textract',
#                           service_name = 's3',
                          region_name = region,
                          aws_access_key_id = KEY,
                          aws_secret_access_key = SECRET)
    response = client.start_document_text_detection(DocumentLocation={
        'S3Object': {'Bucket': s3BucketName,
                     'Name': objectName
                    }
        })
    return response["JobId"]
def isJobComplete(jobId):
 # For production use cases, use SNS based notification 
 # Details at: https://docs.aws.amazon.com/textract/latest/dg/api-async.html
    time.sleep(5)
    client = boto3.client('textract',
#                           service_name = 's3',
                          region_name = region,
                          aws_access_key_id = KEY,
                          aws_secret_access_key = SECRET)
    response = client.get_document_text_detection(JobId=jobId)
    status = response["JobStatus"]
#     print("Job status: {}".format(status))
    while(status == "IN_PROGRESS"):
        time.sleep(5)
        response = client.get_document_text_detection(JobId=jobId)
        status = response["JobStatus"]
#         print("Job status: {}".format(status))
    return status
def getJobResults(jobId):
    pages = []
    client = boto3.client('textract',
#                           service_name = 's3',
                          region_name = region,
                          aws_access_key_id = KEY,
                          aws_secret_access_key = SECRET)
    response = client.get_document_text_detection(JobId=jobId)
    
    pages.append(response)
#     print("Resultset page recieved: {}".format(len(pages)))
    nextToken = None
    if('NextToken' in response):
        nextToken = response['NextToken']
        while(nextToken):
            response = client.get_document_text_detection(JobId=jobId, NextToken=nextToken)
            pages.append(response)
#             print("Resultset page recieved: {}".format(len(pages)))
            nextToken = None
            if('NextToken' in response):
                nextToken = response['NextToken']
    return pages

def save_txt_file(file, file_name):
    f = open(file_name, "w")
    f.write(file)
    f.close()

In [5]:
def AWS_pdf_to_text(key, s3BucketName):
    # Document
    documentName = key
    jobId = startJob(s3BucketName, documentName)
#     print("Started job with id: {}".format(jobId))
    if(isJobComplete(jobId)):
        response = getJobResults(jobId)
    #print(response)
    # Print detected text
    policy = ""
    for resultPage in response:
#         print(resultPage)
        try:
            for item in resultPage["Blocks"]:
                if item["BlockType"] == "LINE":
                    policy = policy + item["Text"] + "\n"
#                     print ('\033[94m' + item["Text"] + '\033[0m')
        except:
            f = open("../output/log_file.txt", 'a')
            f.write(f"The file {key} could not be converted\n")
            f.close()
            
            print(f"The file {key} could not be converted")
    return policy

In [6]:
keyword = "0bc17448527c7798e84937eb897bdaa82c3c4a3c.pdf"
prefix = "english_documents/raw_pdf/"
i = 0


for obj in s3.Bucket(s3BucketName).objects.all().filter(Prefix = prefix):#.:
    if ".pdf" in obj.key and 800 > i > 599: #lower_limit < i < upper_limit:
        print(i, "**", obj.key)
        key = obj.key.replace("raw_pdf", "text_files").replace(".pdf", ".txt")
        policy = AWS_pdf_to_text(obj.key, s3BucketName)
        if policy != "":
            s3.Object(s3BucketName, key).put(Body = policy)
#         save_txt_file(policy, "/home/propietari/Documents/S3/" + obj.key[:-4] + ".txt")
    i += 1

600 ** english_documents/raw_pdf/af0e8494551f905dbe2ef8343336c2d23fff7321.pdf
601 ** english_documents/raw_pdf/af9c7faa78495e24edd4434823fd51f3fce1dc45.pdf
602 ** english_documents/raw_pdf/afb88e3f1cbaf269de2fcce6df00badeb9eab208.pdf
603 ** english_documents/raw_pdf/b0176f37b31691dc4ddefb53e5190b6745fb1d52.pdf
604 ** english_documents/raw_pdf/b0acfe4e509e3ce68f49f1fe2e5a402fa44f04b0.pdf
605 ** english_documents/raw_pdf/b0bf64ca265d27671e5c550556034574f7d10ec7.pdf
606 ** english_documents/raw_pdf/b0f19206368f6bbbafe6b89561d072e8d515fa94.pdf
607 ** english_documents/raw_pdf/b16b1c7501c0c725d2c1bd32a4beb553d779aab0.pdf
608 ** english_documents/raw_pdf/b17aba276d3e856d3f10f2f92f4b35fd4c7d6807.pdf
609 ** english_documents/raw_pdf/b2578ddb534116c76cdefe80945b211169646e6e.pdf
610 ** english_documents/raw_pdf/b2b2a1ccfcf55a4a78b9b3bdf22ecc7120cb8f9e.pdf
611 ** english_documents/raw_pdf/b32bcc6396c09ecae1bd02b7b9f7f531fd69bdad.pdf
612 ** english_documents/raw_pdf/b32f217be5ba3ded6dd5c95a4abedd4

706 ** english_documents/raw_pdf/ca940dce2070d50d9e447b43cba91567d506c3f0.pdf
707 ** english_documents/raw_pdf/ca990bebfb216ebf0575158c31b04fcf25120387.pdf
708 ** english_documents/raw_pdf/cb0bd395cf743bb59e5382750a8fb5917c07a3d3.pdf
709 ** english_documents/raw_pdf/cb57210a6e61a7cfcacba853f38cf7d526fd388a.pdf
710 ** english_documents/raw_pdf/cb6f7e6c4fe04bb06c2c9142690110c8d6b32f3c.pdf
711 ** english_documents/raw_pdf/cbaf851a3748370ae1d85ac9845e3a89d4718e9d.pdf
712 ** english_documents/raw_pdf/cbb79643e1cd64e3f241ef7e790c69a477fc6ed8.pdf
713 ** english_documents/raw_pdf/cbc53c80121c322caad23002c78a04641b391318.pdf
714 ** english_documents/raw_pdf/cbdfa7d523f9ea26673e6bcb7d6a834aa8ff06dd.pdf
715 ** english_documents/raw_pdf/cc1ce3764b4bfad7f7e07b4dd698c83f135d69e5.pdf
716 ** english_documents/raw_pdf/cc85e62e2a2da4b3adf8b10f64d67f955c264de6.pdf
717 ** english_documents/raw_pdf/cd1d83387de243c14732c513b34419510933f669.pdf
718 ** english_documents/raw_pdf/cd22fe54fce84c89f096d770aaf26ac

In [ ]:
policy